## Setup and Dependencies

In [ ]:
# --- Extension Setup ---
%load_ext line_profiler

# --- Module Imports ---
import sys
sys.path.append("..")  # Adjust if your afml repo is nested differently

Working Dir: c:\Users\JoeN\Documents\GitHub\Machine-Learning-Blueprint\notebooks


In [ ]:
import re
import time
import warnings
import winsound
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
)
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

from afml.cache import (
    clear_afml_cache,
    clear_cv_cache,
    get_cache_efficiency_report,
    print_cache_health,
)
from afml.cross_validation import (
    PurgedKFold,
    PurgedSplit,
    analyze_cross_val_scores,
    probability_weighted_accuracy,
)
from afml.data_structures.bars import *
from afml.ensemble.sb_bagging import (
    SequentiallyBootstrappedBaggingClassifier,
    compute_custom_oob_metrics,
    estimate_ensemble_size,
)
from afml.labeling.triple_barrier import (
    add_vertical_barrier,
    get_event_weights,
    triple_barrier_labels,
)
from afml.sample_weights.optimized_attribution import (
    get_weights_by_time_decay_optimized,
)

# from afml.sampling import get_ind_mat_average_uniqueness, get_ind_matrix, seq_bootstrap
from afml.strategies import (
    BollingerStrategy,
    ForexFeatureEngine,
    MACrossoverStrategy,
    create_bollinger_features,
    get_entries,
)
from afml.util import get_daily_vol, value_counts_data

warnings.filterwarnings("ignore")
# plt.style.use("seaborn-v0_8-whitegrid")
plt.style.use("dark_background")plt.style.use("dark_background")

2025-11-10 05:27:21.860 | DEBUG    | afml.cache:<module>:619 - Enhanced cache features available:
2025-11-10 05:27:21.862 | DEBUG    | afml.cache:<module>:620 -   - Robust cache keys for NumPy/Pandas
2025-11-10 05:27:21.870 | DEBUG    | afml.cache:<module>:621 -   - MLflow integration: ✓
2025-11-10 05:27:21.872 | DEBUG    | afml.cache:<module>:622 -   - Backtest caching: ✓
2025-11-10 05:27:21.883 | DEBUG    | afml.cache:<module>:623 -   - Cache monitoring: ✓
2025-11-10 05:27:23.987 | DEBUG    | afml.cache:_configure_numba:59 - Numba cache configured: C:\Users\JoeN\AppData\Local\afml\afml\Cache\numba_cache
2025-11-10 05:27:23.989 | INFO     | afml.cache:initialize_cache_system:308 - AFML cache system initialized:
2025-11-10 05:27:23.995 | INFO     | afml.cache:initialize_cache_system:309 -   Joblib cache: C:\Users\JoeN\AppData\Local\afml\afml\Cache\joblib_cache
2025-11-10 05:27:23.999 | INFO     | afml.cache:initialize_cache_system:310 -   Numba cache: C:\Users\JoeN\AppData\Local\afml\a

In [ ]:
# clear_afml_cache()
# clear_cv_cache()

In [ ]:
# Check cache health anytime
print_cache_health()

# Find functions with low hit rates or high call counts
df = get_cache_efficiency_report()
df.sort_values('calls', ascending=False).head(10)


CACHE HEALTH REPORT

Overall Statistics:
  Total Functions:     11
  Total Calls:         258
  Overall Hit Rate:    43.4%
  Total Cache Size:    0.00 MB

Top Performers (by hit rate):
  1. train_rf: 75.4% (69 calls)
  2. analyze_cross_val_scores: 58.1% (86 calls)
  3. triple_barrier_labels: 23.5% (17 calls)
  4. add_vertical_barrier: 21.1% (19 calls)
  5. create_bollinger_features: 16.7% (6 calls)

Worst Performers (by hit rate):
  1. get_event_weights: 0.0% (33 calls)
  2. trend_scanning_labels: 0.0% (3 calls)
  3. get_bins: 0.0% (7 calls)
  4. drop_labels: 0.0% (7 calls)
  5. calculate_all_features: 0.0% (4 calls)

Recommendations:
  1. Overall hit rate is low (<50%). Consider reviewing cache key generation or function parameter patterns.
  2. Functions with low hit rate: get_event_weights. Review cache key generation for these functions.




,function,calls,hits,misses,hit_rate,avg_time_ms,cache_size_mb,last_access
4,afml.cross_validation.cross_validation.analyze...,86,50,36,58.1%,N/A,N/A,N/A
5,__main__.train_rf,69,52,17,75.4%,N/A,N/A,N/A
3,afml.labeling.triple_barrier.get_event_weights,33,0,33,0.0%,N/A,N/A,N/A
0,afml.labeling.triple_barrier.add_vertical_barrier,19,4,15,21.1%,N/A,N/A,N/A
2,afml.labeling.triple_barrier.triple_barrier_la...,17,4,13,23.5%,N/A,N/A,N/A
7,afml.labeling.triple_barrier.get_events,7,1,6,14.3%,N/A,N/A,N/A
8,afml.labeling.triple_barrier.get_bins,7,0,7,0.0%,N/A,N/A,N/A
9,afml.labeling.triple_barrier.drop_labels,7,0,7,0.0%,N/A,N/A,N/A
1,afml.strategies.bollinger_features.create_boll...,6,1,5,16.7%,N/A,N/A,N/A
10,afml.strategies.ma_crossover_feature_engine.Fo...,4,0,4,0.0%,N/A,N/A,N/A


## 1. Data Preparation

In [ ]:
symbol = "EURUSD"
start_date, end_date = "2018-01-01", "2024-12-31"
sample_start, sample_end = start_date, "2023-12-31"

## 2. Bollinger Band Strategy

In [ ]:
bb_timeframe = "M5"
file = Path(fr"..\data\EURUSD_{bb_timeframe}_time_2018-01-01-2024-12-31.parq")
bb_time_bars = pd.read_parquet(file)

In [ ]:
bb_period, bb_std = 20, 2 # Bollinger Band parameters
bb_strategy = BollingerStrategy(window=bb_period, num_std=bb_std)
bb_lookback = 10
bb_pt_barrier, bb_sl_barrier, bb_time_horizon = (1, 2, dict(days=1))
min_ret = 5e-5
bb_vol_multiplier = 1

### Time-Bars

In [ ]:
bb_side = bb_strategy.generate_signals(bb_time_bars)
bb_df = bb_time_bars.loc[sample_start : sample_end]

print(f"{bb_strategy.get_strategy_name()} Signals:")
value_counts_data(bb_side.reindex(bb_df.index), verbose=True)

# Volatility target for barriers
vol_lookback = 100
vol_target = get_daily_vol(bb_df.close, vol_lookback) * bb_vol_multiplier
close = bb_df.close
_, t_events = get_entries(bb_strategy, bb_df, filter_threshold=vol_target.mean())

vertical_barriers = add_vertical_barrier(t_events, close, **bb_time_horizon)

Bollinger_w20_std2 Signals:

        count  proportion
side                     
 0    373,536    0.842213
-1     35,095    0.079129
 1     34,886    0.078658



2025-11-10 05:27:50.142 | INFO     | afml.filters.filters:cusum_filter:151 - 14,396 CUSUM-filtered events
2025-11-10 05:27:50.243 | INFO     | afml.strategies.signal_processing:get_entries:105 - Bollinger_w20_std2 | 8,143 (11.64%) trade events selected by CUSUM filter (threshold = 0.1612%).


#### Feature Engineering

In [ ]:
bb_feat = create_bollinger_features(bb_time_bars, bb_period, bb_std)
bb_feat_time = bb_feat.copy()
bb_feat_time.info()
# not_stationary = is_stationary(bb_feat_time)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 516825 entries, 2018-01-02 23:20:00 to 2024-12-31 00:00:00
Data columns (total 59 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   spread               516825 non-null  float32
 1   vol                  516825 non-null  float32
 2   h1_vol               516825 non-null  float32
 3   h4_vol               516825 non-null  float32
 4   d1_vol               516825 non-null  float32
 5   ret                  516825 non-null  float32
 6   ret_5                516825 non-null  float32
 7   ret_10               516825 non-null  float32
 8   ret_1_lag_1          516825 non-null  float32
 9   ret_5_lag_1          516825 non-null  float32
 10  ret_10_lag_1         516825 non-null  float32
 11  ret_1_lag_2          516825 non-null  float32
 12  ret_5_lag_2          516825 non-null  float32
 13  ret_10_lag_2         516825 non-null  float32
 14  ret_1_lag_3          516825 non-nu

#### Triple-Barrier Method

In [ ]:
bb_events_tb = triple_barrier_labels(
    close,
    vol_target,
    t_events,
    pt_sl=[bb_pt_barrier, bb_sl_barrier],
    min_ret=min_ret,
    vertical_barrier_times=vertical_barriers,
    side_prediction=bb_side,
    vertical_barrier_zero=True,
    verbose=False,
)

bb_events_tb_time = bb_events_tb.copy()
# bb_events_tb_time_meta = bb_events_tb.copy()
print(f"Triple-Barrier (pt={bb_pt_barrier}, sl={bb_sl_barrier}, h={bb_time_horizon}):")
value_counts_data(bb_events_tb['bin'], verbose=True)

weights = get_event_weights(bb_events_tb, close)
av_uniqueness = weights['tW'].mean()
print(f"Average Uniqueness: {av_uniqueness:.4f}")

Triple-Barrier (pt=1, sl=2, h={'days': 1}):

     count  proportion
bin                   
1    4,800    0.589826
0    3,338    0.410174

Average Uniqueness: 0.5632


#### CV of Weighting Methods

In [ ]:
from os import cpu_count

# Reserve 1 CPU if you want to do something else during training, otherwise set to -1
N_JOBS = cpu_count() - 1
N_ESTIMATORS = 100
seed = 7
min_w_leaf = 0.05
max_depth = 4
n_splits = 3
pct_embargo = 0.01
test_size = 0.2

In [ ]:
cont = bb_events_tb_time.copy()
X = bb_feat_time.reindex(cont.index)
y = cont["bin"]
t1 = cont["t1"]

train, test = PurgedSplit(t1, test_size).split(X)
X_train, X_test, y_train, y_test = (
        X.iloc[train],
        X.iloc[test],
        y.iloc[train],
        y.iloc[test],
    )

cont_train = cont.iloc[train]
cont_train = get_event_weights(cont_train, bb_df.close)
bb_cont_train = cont_train.copy()

cv_gen = PurgedKFold(n_splits, cont_train["t1"], pct_embargo)

In [ ]:
avg_u = cont_train.tW.mean()
print(f"Average Uniqueness in Training Set: {avg_u:.4f}")

weighting_schemes = {
    "unweighted": pd.Series(1., index=cont_train.index),
    "uniqueness": cont_train["tW"],
    "return": cont_train["w"],
    }

decay_factors = [0.0, 0.25, 0.5, 0.75]
time_decay_weights = {}
for time_decay in decay_factors:
    decay_w = get_weights_by_time_decay_optimized(
                triple_barrier_events=cont_train,
                close_index=close.index,
                last_weight=time_decay,
                linear=True,
                av_uniqueness=cont_train["tW"],
            )
    time_decay_weights[f"decay_{time_decay}"] = decay_w
        
weighting_schemes.keys()

Average Uniqueness in Training Set: 0.5623


dict_keys(['unweighted', 'uniqueness', 'return'])

##### Selection of Best Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest

clf = RandomForestClassifier(
    criterion='entropy',
    n_estimators=N_ESTIMATORS,
    class_weight="balanced_subsample",
    max_samples=avg_u,
    min_weight_fraction_leaf=min_w_leaf,
    max_depth=max_depth,
    random_state=seed,
    n_jobs=N_JOBS,  # Use all available cores
    )


- Analyze all CV scores for all weighting schemes to find the best scheme

In [ ]:
all_cv_scores_df = pd.DataFrame()
all_cv_scores_d = {}
all_cms = {}
best_score, best_scheme = None, None

if set(y_train.values) == {0, 1}:
    scoring = "f1"  # f1 for meta-labeling
else:
    scoring = "neg_log_loss"  # symmetric towards all cases

for scheme, w in tqdm(weighting_schemes.items()):
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        clf, X_train, y_train, cv_gen, 
        sample_weight_train=w, 
        sample_weight_score=w,
    )
    all_cms[scheme] = cms
    all_cv_scores_d[scheme] = cv_scores
    score = cv_scores[scoring].mean()
    recall = cv_scores_df.loc["recall", "mean"]
    recall_std = cv_scores_df.loc["recall", "std"]
    for idx, row in cv_scores_df.iterrows():
        all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if scoring == "f1" and (np.allclose([recall + recall_std], [1.0], 0.025) or np.allclose([recall - recall_std], [0.0], 0.025)):
        print(f"Recall score ({all_cv_scores_df.loc['recall', scheme]}) collapses for {scheme} weighting scheme")
        continue
    best_score = max(best_score, score) if best_score is not None else score
    if score == best_score:
        best_scheme = scheme

print(f"{best_scheme.title()} is the best weighting scheme with {scoring} = {best_score:.4f}")
print("\nWeighting Scheme CV:")
all_cv_scores_df

  0%|          | 0/3 [00:00<?, ?it/s]2025-11-10 05:27:52.226 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for analyze_cross_val_scores
2025-11-10 05:27:52.244 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for analyze_cross_val_scores
2025-11-10 05:27:52.260 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for analyze_cross_val_scores
100%|██████████| 3/3 [00:00<00:00, 61.82it/s]

Recall score (0.0000 ± 0.0000) collapses for return weighting scheme
Uniqueness is the best weighting scheme with f1 = 0.6279

Weighting Scheme CV:


,unweighted,uniqueness,return
accuracy,0.5306 ± 0.0116,0.5370 ± 0.0140,0.6243 ± 0.0071
pwa,0.5406 ± 0.0130,0.5545 ± 0.0049,0.6275 ± 0.0061
neg_log_loss,-0.6912 ± 0.0014,-0.6896 ± 0.0004,-0.6723 ± 0.0049
precision,0.6087 ± 0.0098,0.6039 ± 0.0201,0.0000 ± 0.0000
recall,0.5726 ± 0.0312,0.6549 ± 0.0202,0.0000 ± 0.0000
f1,0.5897 ± 0.0177,0.6279 ± 0.0107,0.0000 ± 0.0000


- Test if time-decay improves performance of best model

In [ ]:
best_model_decay_cv_scores = pd.DataFrame()

for scheme, decay_factor in tqdm(time_decay_weights.items()):
    best_scheme_o = best_scheme.split("_decay")[0]
    sample_weight = weighting_schemes[best_scheme_o] * decay_factor
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weight, 
        sample_weight_score=sample_weight,
    )
    score = cv_scores[scoring].mean()
    best_score = max(best_score, score) if best_score is not None else score
    scheme = f"{best_scheme_o}_{scheme}"
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    for idx, row in cv_scores_df.iterrows():
        best_model_decay_cv_scores.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if score == best_score:
        best_scheme = scheme
        weighting_schemes[best_scheme] = sample_weight
    all_cv_scores_df[scheme] = best_model_decay_cv_scores[scheme]
best_model_decay_cv_scores[f"{best_scheme_o}_decay_1.0"] = all_cv_scores_df[best_scheme_o]
        
print(f"\n{best_scheme.title()} model achieved the best {scoring} score of {best_score:.4f}")
best_model_decay_cv_scores

 25%|██▌       | 1/4 [00:00<00:00,  8.62it/s]2025-11-10 05:27:52.587 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for analyze_cross_val_scores
2025-11-10 05:27:52.636 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for analyze_cross_val_scores
100%|██████████| 4/4 [00:00<00:00, 14.76it/s]



Uniqueness_Decay_0.25 model achieved the best f1 score of 0.6317


,uniqueness_decay_0.0,uniqueness_decay_0.25,uniqueness_decay_0.5,uniqueness_decay_0.75,uniqueness_decay_1.0
accuracy,0.5351 ± 0.0163,0.5416 ± 0.0125,0.5387 ± 0.0159,0.5350 ± 0.0142,0.5370 ± 0.0140
pwa,0.5537 ± 0.0145,0.5547 ± 0.0056,0.5543 ± 0.0060,0.5517 ± 0.0042,0.5545 ± 0.0049
neg_log_loss,-0.6898 ± 0.0014,-0.6897 ± 0.0005,-0.6898 ± 0.0005,-0.6899 ± 0.0004,-0.6896 ± 0.0004
precision,0.5995 ± 0.0156,0.6064 ± 0.0175,0.6042 ± 0.0185,0.6024 ± 0.0198,0.6039 ± 0.0201
recall,0.6607 ± 0.0322,0.6601 ± 0.0226,0.6577 ± 0.0210,0.6522 ± 0.0210,0.6549 ± 0.0202
f1,0.6281 ± 0.0173,0.6317 ± 0.0110,0.6295 ± 0.0138,0.6258 ± 0.0103,0.6279 ± 0.0107


##### Sequential Bootstrap

In [ ]:
# Random Forest default of max_features is sqrt, 
# which means I don't have to calculate it.
base_rf = clone(clf).set_params(
    n_estimators=1,
    bootstrap=False,
    n_jobs=None,
    max_samples=None,
    random_state=None,
    )

seq_rf = SequentiallyBootstrappedBaggingClassifier(
    samples_info_sets=cont_train.t1,
    price_bars_index=bb_df.index,
    estimator=base_rf,
    n_estimators=N_ESTIMATORS, # set low to save time
    max_samples=avg_u, # Set to average uniqueness
    oob_score=True,
    n_jobs=N_JOBS,
    random_state=seed,
    verbose=False,
)
seq_rf

SequentiallyBootstrappedBaggingClassifier(estimator=RandomForestClassifier(bootstrap=False,
                                                                           class_weight='balanced_subsample',
                                                                           criterion='entropy',
                                                                           max_depth=4,
                                                                           min_weight_fraction_leaf=0.05,
                                                                           n_estimators=1),
                                          max_samples=0.5622874569199091,
                                          n_estimators=100, n_jobs=3,
                                          oob_score=True,
                                          price_bars_index=DatetimeIndex(['2018-01-01 23:05:00', '2018-01-01 23:10:00',
               '...
2018-01-03 11:35:00   2018-01-04 11:05:00
2018-01-03 15:00:00   2018-01-04 15:00:00
2018-01-03 16:05:00   2018-01-04 10:40:00
2018-01-03 20:10:00   2018-01-03 21:35:00
                              ...        
2022-11-04 03:40:00   2022-11-04 15:00:00
2022-11-04 06:15:00   2022-11-04 15:00:00
2022-11-04 09:15:00   2022-11-04 13:15:00
2022-11-04 09:55:00   2022-11-04 11:35:00
2022-11-04 14:25:00   2022-11-04 15:00:00
Name: t1, Length: 6506, dtype: datetime64[ns],
                                          verbose=False)

In [ ]:
from afml.cache.cv_cache import cv_cacheable


@cv_cacheable
def train_rf(classifier, X, y, sample_weight=None):
    time0 = time.time()
    clf = clone(classifier).set_params(oob_score=True).fit(X, y, sample_weight)
    time1 = pd.Timedelta(seconds=time.time() - time0).round('1s')
    print(f"{clf.__class__.__name__} trained in {time1}.")
    return clf

In [ ]:
w = weighting_schemes[best_scheme]
rf = clone(clf).set_params(oob_score=True)
seq_rf1 = clone(seq_rf).set_params(max_samples=1.0)

rf = train_rf(rf, X_train, y_train, w)
seq_rf = train_rf(seq_rf, X_train, y_train, w)
seq_rf1 = train_rf(seq_rf1, X_train, y_train, w)

ensembles = {
    "standard_rf": rf,
    "sequential_rf": seq_rf,  # max_samples=avg_u
    "sequential_rf_all": seq_rf1,  # max_samples=1.0
}

if not best_scheme.startswith("unweighted"):
    print(f"Training: Sequential Bootstrap (max_samples=avg_u) - Unweighted...")
    seq_rfu = train_rf(clone(seq_rf), X_train, y_train)  # max_samples=avg_u
    ensembles["sequential_rf_unweighted"] = seq_rfu

    print(f"Training: Sequential Bootstrap (max_samples=1.0) - Unweighted...")
    seq_rfu1 = train_rf(clone(seq_rf1), X_train, y_train)  # max_samples=avg_u
    ensembles["sequential_rf_unweighted_all"] = seq_rfu1

scoring_methods = {
            "f1": f1_score,
            "precision": precision_score,
            "recall": recall_score,
            "neg_log_loss": log_loss,
            "pwa": probability_weighted_accuracy,
            "accuracy": accuracy_score,
        }
all_scores_oos = pd.DataFrame()

for name, classifier in ensembles.items():
    prob = classifier.predict_proba(X_test)[:, 1]
    pred = (prob > 0.5).astype(int)
    for method, scoring in scoring_methods.items():
        y_pred = prob if scoring in (probability_weighted_accuracy, log_loss) else pred
        score = scoring(y_test, y_pred)
        if method == "neg_log_loss":
            score *= -1
        all_scores_oos.loc[method, name] = score
    all_scores_oos.loc["oob_accuracy", name] = classifier.oob_score_
    all_scores_oos.loc["oob_test_gap", name] = abs(all_scores_oos.loc["accuracy", name] - classifier.oob_score_)

print(f"Weighting scheme: {best_scheme}")
print(f"\nAverage uniqueness = {avg_u:.4f}\n")

bb_all_scores_oos = all_scores_oos.copy()

# winsound.Beep(1000, 1000) # Alert

all_scores_oos.round(4)

2025-11-10 05:27:52.982 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for train_rf
2025-11-10 05:27:53.145 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for train_rf


2025-11-10 05:27:53.331 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for train_rf
2025-11-10 05:27:53.473 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for train_rf


Training: Sequential Bootstrap (max_samples=avg_u) - Unweighted...
Training: Sequential Bootstrap (max_samples=1.0) - Unweighted...


2025-11-10 05:27:53.644 | INFO     | afml.cache.cv_cache:wrapper:230 - CV cache hit for train_rf


Weighting scheme: uniqueness_decay_0.25

Average uniqueness = 0.5623



,standard_rf,sequential_rf,sequential_rf_all,sequential_rf_unweighted,sequential_rf_unweighted_all
f1,0.6157,0.6306,0.6409,0.5796,0.5759
precision,0.5923,0.5938,0.5991,0.5996,0.5989
recall,0.6410,0.6722,0.6889,0.5609,0.5546
neg_log_loss,-0.6899,-0.6895,-0.6897,-0.6912,-0.6923
pwa,0.5531,0.5566,0.5556,0.5391,0.5254
accuracy,0.5274,0.5347,0.5439,0.5194,0.5175
oob_accuracy,0.5372,0.5389,0.5403,0.5324,0.5269
oob_test_gap,0.0098,0.0042,0.0037,0.0131,0.0094


#### Conclusion

**Weighting scheme**: Average uniqueness with linear decay (last_weight=0.25)

| Metric | standard_rf | sequential_rf | sequential_rf_all | sequential_rf_unweighted | sequential_rf_unweighted_all |
|---|---:|---:|---:|---:|---:|
| f1 | 0.6157 | 0.6306 | **0.6409** | 0.5796 | 0.5759 |
| recall | 0.6410 | 0.6722 | **0.6889** | 0.5609 | 0.5546 |
| precision | 0.5923 | 0.5938 | 0.5991 | 0.5996 | 0.5989 |
| oob_test_gap | 0.0098 | **0.0042** | 0.0037 | 0.0131 | 0.0094 |

**Training Times:**
- standard_rf (weighted, avg_u): **5 seconds**
- sequential_rf (weighted, avg_u): **7 minutes 8 seconds**
- sequential_rf_all (weighted, max_samples=1.0): **12 minutes 50 seconds**
- sequential_rf_unweighted (unweighted, avg_u): **8 minutes 40 seconds**  
- sequential_rf_unweighted_all (unweighted, max_samples=1.0): **13 minutes 39 seconds**


##### Meta-Labeling Strategic Assessment:

**For meta-labeling applications where F1 and recall are paramount, sequential_rf_all emerges as the optimal choice** despite the 80% training time increase. Here's the strategic rationale:

1. **F1 Performance Justifies Computational Cost**: 
   - The +1.6% F1 improvement (0.6306 → 0.6409) may appear modest, but in meta-labeling context this represents **meaningful edge enhancement**
   - The additional 5 minutes 42 seconds of training time is trivial for a production model that will be deployed for weeks/months
   - Meta-labeling models are typically retrained infrequently, making computational efficiency less critical than performance

2. **Recall Advantage is Strategically Significant**:
   - sequential_rf_all achieves the highest recall (0.6889), which is crucial for meta-labeling
   - Higher recall means capturing more profitable secondary signals from your primary MA crossover strategy
   - The +2.5% recall improvement over sequential_rf directly impacts strategy capacity

3. **Generalization Remains Excellent**:
   - sequential_rf_all maintains superb generalization (OOB gap: 0.0037)
   - The minimal overfitting risk supports deployment confidence
   - All sequential models outperform standard_rf on generalization metrics

4. **Weighted Models Demonstrate Clear Superiority**:
   - Weighted sequential models outperform unweighted by **+8.8% F1** for avg_u and **+11.3% F1** for 1.0
   - This confirms sample weighting's critical role in capturing temporal dependencies for financial data

##### Strategic Recommendation for Meta-Labeling:

**Deploy sequential_rf_all (weighted, max_samples=1.0)** with the following workflow:

- **Research Phase**: Use sequential_rf (weighted, avg_u) for rapid iteration (7:08 training time)
- **Production Deployment**: Use sequential_rf_all (weighted, max_samples=1.0) for final models (12:50 training time)
- **Avoid Unweighted Models**: The performance degradation isn't justified by slightly faster training

**Bottom Line**: In meta-labeling, where filtering quality directly impacts strategy profitability, the F1 and recall advantages of sequential_rf_all justify the modest training time increase. The 80% longer training is an acceptable tradeoff for enhanced signal filtering capability in a production trading system.

## 3. Moving Average Crossover Strategy

In [ ]:
from afml.strategies.ma_crossover_feature_engine import ForexFeatureEngine

ma_timeframe = "M15"
file = Path(fr"..\data\EURUSD_{ma_timeframe}_time_2018-01-01-2024-12-31.parq")
ma_time_bars = pd.read_parquet(file)

fast_window, slow_window = 50, 200
ma_strategy = MACrossoverStrategy(fast_window, slow_window)
ma_pt_barrier, ma_sl_barrier, ma_time_horizon = (0, 2, dict(days=3))
ma_vol_multiplier = 1

### Time-Bars

In [ ]:
ma_side = ma_strategy.generate_signals(ma_time_bars)
ma_df = ma_time_bars.loc[sample_start : sample_end]


print(f"{ma_strategy.get_strategy_name()} Signals:")
value_counts_data(ma_side.reindex(ma_df.index), verbose=True)

# Volatility target for barriers
vol_lookback = 100
vol_target = get_daily_vol(ma_df.close, vol_lookback) * ma_vol_multiplier
close = ma_df.close

thres = vol_target.mean()
_, t_events = get_entries(ma_strategy, ma_df, filter_threshold=vol_target.mean())

vertical_barriers = add_vertical_barrier(t_events, close, **ma_time_horizon)

2025-11-10 05:28:02.900 | INFO     | afml.filters.filters:cusum_filter:151 - 5,301 CUSUM-filtered events


MACrossover_50_200 Signals:

       count  proportion
side                    
-1    75,984    0.513940
 1    71,663    0.484714
 0       199    0.001346



2025-11-10 05:28:02.947 | INFO     | afml.strategies.signal_processing:get_entries:105 - MACrossover_50_200 | 5,295 (3.59%) trade events selected by CUSUM filter (threshold = 0.2606%).


#### Feature Engineering

In [ ]:
ma_feat_engine = ForexFeatureEngine(pair_name=symbol)
ma_feat_time = ma_feat_engine.calculate_all_features(ma_time_bars, ma_timeframe, lr_period=(5, 20))
ma_feat_time.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 172386 entries, 2018-01-01 23:15:00 to 2024-12-31 00:00:00
Data columns (total 94 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ma_10                           172386 non-null  float32
 1   ma_20                           172386 non-null  float32
 2   ma_50                           172386 non-null  float32
 3   ma_100                          172386 non-null  float32
 4   ma_200                          172386 non-null  float32
 5   ma_10_20_cross                  172386 non-null  float64
 6   ma_20_50_cross                  172386 non-null  float64
 7   ma_50_200_cross                 172386 non-null  float64
 8   ma_spread_10_20                 172386 non-null  float32
 9   ma_spread_20_50                 172386 non-null  float32
 10  ma_spread_50_200                172386 non-null  float32
 11  ma_20_slope                     172386 non-n

In [ ]:
for i, col in enumerate(ma_feat_time):
    print(f"{i:>3}. {col}")

  0. ma_10
  1. ma_20
  2. ma_50
  3. ma_100
  4. ma_200
  5. ma_10_20_cross
  6. ma_20_50_cross
  7. ma_50_200_cross
  8. ma_spread_10_20
  9. ma_spread_20_50
 10. ma_spread_50_200
 11. ma_20_slope
 12. ma_50_slope
 13. price_above_ma_20
 14. price_above_ma_50
 15. ma_ribbon_aligned
 16. atr_14
 17. atr_21
 18. atr_regime
 19. realized_vol_10
 20. realized_vol_20
 21. realized_vol_50
 22. vol_of_vol
 23. hl_range
 24. hl_range_ma
 25. hl_range_regime
 26. bb_upper
 27. bb_lower
 28. bb_percent
 29. bb_bandwidth
 30. bb_squeeze
 31. efficiency_ratio_14
 32. efficiency_ratio_30
 33. adx_14
 34. dmp_14
 35. dmn_14
 36. adx_trend_strength
 37. adx_trend_direction
 38. trend_window
 39. trend_slope
 40. trend_t_value
 41. trend_rsquared
 42. trend_ret
 43. roc_10
 44. roc_20
 45. momentum_14
 46. hh_ll_20
 47. trend_persistence
 48. return_skew_20
 49. return_kurtosis_20
 50. var_95
 51. cvar_95
 52. market_stress
 53. current_drawdown
 54. days_since_high
 55. hour_sin_h1
 56. hour_cos_h1

#### Triple-Barrier Method

In [ ]:
ma_events_tb = triple_barrier_labels(
    close=close,
    target=vol_target,
    t_events=t_events,
    pt_sl=[ma_pt_barrier, ma_sl_barrier],
    min_ret=min_ret,
    vertical_barrier_times=vertical_barriers,
    side_prediction=ma_side,
    vertical_barrier_zero=False,
    verbose=False,
)
ma_events_tb_time = ma_events_tb.copy()
ma_events_tb.info()

print(f"Triple-Barrier (pt={ma_pt_barrier}, sl={ma_sl_barrier}, h={ma_time_horizon}):")
value_counts_data(ma_events_tb.bin, verbose=True)

weights = get_event_weights(ma_events_tb, close)
av_uniqueness = weights['tW'].mean()
print(f"Average Uniqueness: {av_uniqueness:.4f}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5288 entries, 2018-01-04 11:00:00 to 2023-12-28 16:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   t1      5288 non-null   datetime64[ns]
 1   trgt    5288 non-null   float64       
 2   ret     5288 non-null   float32       
 3   bin     5288 non-null   int8          
 4   side    5288 non-null   int8          
dtypes: datetime64[ns](1), float32(1), float64(1), int8(2)
memory usage: 154.9 KB
Triple-Barrier (pt=0, sl=2, h={'days': 3}):

     count  proportion
bin                   
0    3,003     0.56789
1    2,285     0.43211

Average Uniqueness: 0.1931


#### CV of Weighting Methods

In [ ]:
from os import cpu_count

# Reserve 1 CPU if you want to do something else during training, otherwise set to -1
N_JOBS = cpu_count() - 1
N_ESTIMATORS = 100
seed = 7
min_w_leaf = 0.05
max_depth = 4
n_splits = 3
pct_embargo = 0.01
test_size = 0.2

In [ ]:
cont = ma_events_tb_time.copy()
X = ma_feat_time.reindex(cont.index)
y = cont["bin"]
t1 = cont["t1"]

train, test = PurgedSplit(t1, test_size).split(X)
X_train, X_test, y_train, y_test = (
        X.iloc[train],
        X.iloc[test],
        y.iloc[train],
        y.iloc[test],
    )

cont_train = cont.iloc[train]
cont_train = get_event_weights(cont_train, bb_df.close)
bb_cont_train = cont_train.copy()

cv_gen = PurgedKFold(n_splits, cont_train["t1"], pct_embargo)

In [ ]:
avg_u = cont_train.tW.mean()
print(f"Average Uniqueness in Training Set: {avg_u:.4f}")

weighting_schemes = {
    "unweighted": pd.Series(1., index=cont_train.index),
    "uniqueness": cont_train["tW"],
    "return": cont_train["w"],
    }

decay_factors = [0.0, 0.25, 0.5, 0.75]
time_decay_weights = {}
for time_decay in decay_factors:
    decay_w = get_weights_by_time_decay_optimized(
                triple_barrier_events=cont_train,
                close_index=close.index,
                last_weight=time_decay,
                linear=True,
                av_uniqueness=cont_train["tW"],
            )
    time_decay_weights[f"decay_{time_decay}"] = decay_w
        
weighting_schemes.keys()

Average Uniqueness in Training Set: 0.1983


dict_keys(['unweighted', 'uniqueness', 'return'])

##### Selection of Best Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest

clf = RandomForestClassifier(
    criterion='entropy',
    n_estimators=N_ESTIMATORS,
    class_weight="balanced_subsample",
    max_samples=avg_u,
    min_weight_fraction_leaf=min_w_leaf,
    max_depth=max_depth,
    random_state=seed,
    n_jobs=N_JOBS,  # Use all available cores
    )


- Analyze all CV scores for all weighting schemes to find the best scheme

In [ ]:
all_cv_scores_df = pd.DataFrame()
all_cv_scores_d = {}
all_cms = {}
best_score, best_scheme = None, None

if set(y_train.values) == {0, 1}:
    scoring = "f1"  # f1 for meta-labeling
else:
    scoring = "neg_log_loss"  # symmetric towards all cases

for scheme, w in tqdm(weighting_schemes.items()):
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        clf, X_train, y_train, cv_gen, 
        sample_weight_train=w, 
        sample_weight_score=w,
    )
    all_cms[scheme] = cms
    all_cv_scores_d[scheme] = cv_scores
    score = cv_scores[scoring].mean()
    recall = cv_scores_df.loc["recall", "mean"]
    recall_std = cv_scores_df.loc["recall", "std"]
    for idx, row in cv_scores_df.iterrows():
        all_cv_scores_df.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if scoring == "f1" and (np.allclose([recall + recall_std], [1.0], 0.025) or np.allclose([recall - recall_std], [0.0], 0.025)):
        print(f"Recall score ({all_cv_scores_df.loc['recall', scheme]}) collapses for {scheme} weighting scheme")
        continue
    best_score = max(best_score, score) if best_score is not None else score
    if score == best_score:
        best_scheme = scheme

print(f"{best_scheme.title()} is the best weighting scheme with {scoring} = {best_score:.4f}")
print("\nWeighting Scheme CV:")
all_cv_scores_df

  0%|          | 0/3 [00:00<?, ?it/s]2025-11-10 05:28:12.137 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for analyze_cross_val_scores - computing...
2025-11-10 05:28:16.374 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: fe767a5adcde41fff86735f29126f816
 33%|███▎      | 1/3 [00:04<00:08,  4.26s/it]2025-11-10 05:28:16.407 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for analyze_cross_val_scores - computing...
2025-11-10 05:28:18.997 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: da9e29fa8f578eb04a47a281a843f111
 67%|██████▋   | 2/3 [00:06<00:03,  3.29s/it]2025-11-10 05:28:19.010 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for analyze_cross_val_scores - computing...
2025-11-10 05:28:22.614 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: e8594f32d8580bfb88c15e74faf792e5
100%|██████████| 3/3 [00:10<00:00,  3.50s/it]

Recall score (0.8095 ± 0.1875) collapses for return weighting scheme
Unweighted is the best weighting scheme with f1 = 0.4901

Weighting Scheme CV:


,unweighted,uniqueness,return
accuracy,0.4853 ± 0.0270,0.5943 ± 0.0124,0.5021 ± 0.0244
pwa,0.4821 ± 0.0436,0.6294 ± 0.0165,0.4893 ± 0.0170
neg_log_loss,-0.7070 ± 0.0171,-0.6755 ± 0.0035,-0.7048 ± 0.0023
precision,0.4444 ± 0.0139,0.4740 ± 0.0103,0.5033 ± 0.0231
recall,0.5750 ± 0.1610,0.2339 ± 0.0814,0.8095 ± 0.1875
f1,0.4901 ± 0.0478,0.3056 ± 0.0770,0.6140 ± 0.0750


- Test if time-decay improves performance of best model

In [ ]:
best_model_decay_cv_scores = pd.DataFrame()

for scheme, decay_factor in tqdm(time_decay_weights.items()):
    best_scheme_o = best_scheme.split("_decay")[0]
    sample_weight = weighting_schemes[best_scheme_o] * decay_factor
    cv_scores, cv_scores_df, cms = analyze_cross_val_scores(
        clf, X_train, y_train, cv_gen, 
        sample_weight_train=sample_weight, 
        sample_weight_score=sample_weight,
    )
    score = cv_scores[scoring].mean()
    best_score = max(best_score, score) if best_score is not None else score
    scheme = f"{best_scheme_o}_{scheme}"
    all_cv_scores_d[scheme] = cv_scores
    all_cms[scheme] = cms
    for idx, row in cv_scores_df.iterrows():
        best_model_decay_cv_scores.loc[idx, scheme] = f"{row['mean']:.4f} ± {row['std']:.4f}"
    if score == best_score:
        best_scheme = scheme
        weighting_schemes[best_scheme] = sample_weight
    all_cv_scores_df[scheme] = best_model_decay_cv_scores[scheme]
best_model_decay_cv_scores[f"{best_scheme_o}_decay_1.0"] = all_cv_scores_df[best_scheme_o]
        
print(f"\n{best_scheme.title()} model achieved the best {scoring} score of {best_score:.4f}")
best_model_decay_cv_scores

  0%|          | 0/4 [00:00<?, ?it/s]2025-11-10 05:28:22.726 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for analyze_cross_val_scores - computing...
2025-11-10 05:28:26.018 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: 1f6d2c9e17356961c062cddb62bfe612
 25%|██▌       | 1/4 [00:03<00:09,  3.31s/it]2025-11-10 05:28:26.033 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for analyze_cross_val_scores - computing...
2025-11-10 05:28:28.899 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: 0be88fe1f1421de5f3309b0375d38b67
 50%|█████     | 2/4 [00:06<00:06,  3.06s/it]2025-11-10 05:28:28.913 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for analyze_cross_val_scores - computing...
2025-11-10 05:28:32.992 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: 2cb021dc26c1a3f8701f03e4c2685a5e
 75%|███████▌  | 3/4 [00:10<00:03,  3.54s/it]2025-11-10 05:28:33.030 | INFO     | afml.cache.cv_cache:wrapper:238 - CV c


Unweighted model achieved the best f1 score of 0.4901


,unweighted_decay_0.0,unweighted_decay_0.25,unweighted_decay_0.5,unweighted_decay_0.75,unweighted_decay_1.0
accuracy,0.4975 ± 0.0306,0.4936 ± 0.0280,0.4804 ± 0.0289,0.4835 ± 0.0360,0.4853 ± 0.0270
pwa,0.4895 ± 0.0483,0.4839 ± 0.0432,0.4757 ± 0.0498,0.4802 ± 0.0480,0.4821 ± 0.0436
neg_log_loss,-0.7026 ± 0.0127,-0.7044 ± 0.0137,-0.7079 ± 0.0178,-0.7071 ± 0.0172,-0.7070 ± 0.0171
precision,0.4343 ± 0.0062,0.4395 ± 0.0127,0.4338 ± 0.0101,0.4408 ± 0.0187,0.4444 ± 0.0139
recall,0.4447 ± 0.2221,0.4820 ± 0.1679,0.5324 ± 0.1805,0.5534 ± 0.1726,0.5750 ± 0.1610
f1,0.4124 ± 0.0993,0.4448 ± 0.0615,0.4641 ± 0.0606,0.4776 ± 0.0554,0.4901 ± 0.0478


##### Sequential Bootstrap

In [ ]:
# Random Forest default of max_features is sqrt, 
# which means I don't have to calculate it.
base_rf = clone(clf).set_params(
    n_estimators=1,
    bootstrap=False,
    n_jobs=None,
    max_samples=None,
    random_state=None,
    )

seq_rf = SequentiallyBootstrappedBaggingClassifier(
    samples_info_sets=cont_train.t1,
    price_bars_index=bb_df.index,
    estimator=base_rf,
    n_estimators=N_ESTIMATORS, # set low to save time
    max_samples=avg_u, # Set to average uniqueness
    oob_score=True,
    n_jobs=N_JOBS,
    random_state=seed,
    verbose=False,
)
seq_rf

SequentiallyBootstrappedBaggingClassifier(estimator=RandomForestClassifier(bootstrap=False,
                                                                           class_weight='balanced_subsample',
                                                                           criterion='entropy',
                                                                           max_depth=4,
                                                                           min_weight_fraction_leaf=0.05,
                                                                           n_estimators=1),
                                          max_samples=0.19827683983120375,
                                          n_estimators=100, n_jobs=3,
                                          oob_score=True,
                                          price_bars_index=DatetimeIndex(['2018-01-01 23:05:00', '2018-01-01 23:10:00',...
2018-01-04 15:45:00   2018-01-08 00:15:00
2018-01-05 09:30:00   2018-01-08 09:00:00
2018-01-05 15:30:00   2018-01-08 15:30:00
2018-01-08 06:45:00   2018-01-11 06:45:00
                              ...        
2022-11-11 15:15:00   2022-11-14 15:15:00
2022-11-11 15:45:00   2022-11-14 15:45:00
2022-11-11 16:15:00   2022-11-14 16:15:00
2022-11-11 18:00:00   2022-11-14 18:00:00
2022-11-11 21:00:00   2022-11-14 21:00:00
Name: t1, Length: 4206, dtype: datetime64[ns],
                                          verbose=False)

In [ ]:
from afml.cache.cv_cache import cv_cacheable


@cv_cacheable
def train_rf(classifier, X, y, sample_weight=None):
    time0 = time.time()
    clf = clone(classifier).set_params(oob_score=True).fit(X, y, sample_weight)
    time1 = pd.Timedelta(seconds=time.time() - time0).round('1s')
    print(f"{clf.__class__.__name__} trained in {time1}.")
    return clf

In [ ]:
w = weighting_schemes[best_scheme]
rf = clone(clf).set_params(oob_score=True)
seq_rf1 = clone(seq_rf).set_params(max_samples=1.0)

rf = train_rf(rf, X_train, y_train, w)
seq_rf = train_rf(seq_rf, X_train, y_train, w)
seq_rf1 = train_rf(seq_rf1, X_train, y_train, w)

ensembles = {
    "standard_rf": rf,
    "sequential_rf": seq_rf,  # max_samples=avg_u
    "sequential_rf_all": seq_rf1,  # max_samples=1.0
}

if best_scheme != "unweighted":
    print(f"Training: Sequential Bootstrap (max_samples=avg_u) - Unweighted...")
    seq_rfu = train_rf(clone(seq_rf), X_train, y_train)  # max_samples=avg_u
    ensembles["sequential_rf_unweighted"] = seq_rfu

    print(f"Training: Sequential Bootstrap (max_samples=1.0) - Unweighted...")
    seq_rfu1 = train_rf(clone(seq_rf1), X_train, y_train)  # max_samples=avg_u
    ensembles["sequential_rf_unweighted_all"] = seq_rfu1

scoring_methods = {
            "f1": f1_score,
            "precision": precision_score,
            "recall": recall_score,
            "neg_log_loss": log_loss,
            "pwa": probability_weighted_accuracy,
            "accuracy": accuracy_score,
        }

all_scores_oos = pd.DataFrame()

for name, classifier in ensembles.items():
    prob = classifier.predict_proba(X_test)[:, 1]
    pred = (prob > 0.5).astype(int)
    for method, scoring in scoring_methods.items():
        y_pred = prob if scoring in (probability_weighted_accuracy, log_loss) else pred
        score = scoring(y_test, y_pred)
        if method == "neg_log_loss":
            score *= -1
        all_scores_oos.loc[method, name] = score
    all_scores_oos.loc["oob_accuracy", name] = classifier.oob_score_
    all_scores_oos.loc["oob_test_gap", name] = abs(all_scores_oos.loc["accuracy", name] - classifier.oob_score_)

print(f"Weighting scheme: {best_scheme}")
print(f"\nAverage uniqueness = {avg_u:.4f}\n")
ma_all_scores_oos = all_scores_oos.copy()

# winsound.Beep(1000, 1000) # Alert

all_scores_oos.round(4)

2025-11-10 05:28:37.549 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for train_rf - computing...
2025-11-10 05:28:39.721 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: e04818e92360851a9812fcec94a63ced
2025-11-10 05:28:39.725 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for train_rf - computing...


RandomForestClassifier trained in 0 days 00:00:02.


2025-11-10 05:33:40.202 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: 86a849cc53980a42307c6a67a0fe3631
2025-11-10 05:33:40.222 | INFO     | afml.cache.cv_cache:wrapper:238 - CV cache miss for train_rf - computing...


SequentiallyBootstrappedBaggingClassifier trained in 0 days 00:05:00.


2025-11-10 06:04:22.016 | DEBUG    | afml.cache.cv_cache:wrapper:245 - Cached CV result: 997ebe86633e184ba442bfae8fa07842


SequentiallyBootstrappedBaggingClassifier trained in 0 days 00:30:42.
Weighting scheme: unweighted

Average uniqueness = 0.1983



,standard_rf,sequential_rf,sequential_rf_all
f1,0.3639,0.4019,0.4573
precision,0.3947,0.3832,0.4112
recall,0.3375,0.4225,0.5150
neg_log_loss,-0.6858,-0.6877,-0.6881
pwa,0.5837,0.5671,0.5665
accuracy,0.5535,0.5241,0.5374
oob_accuracy,0.5568,0.5635,0.5604
oob_test_gap,0.0034,0.0394,0.0230


#### Conclusion

In meta-labeling, we're specifically trying to filter false signals and improve the precision of a primary strategy, making F1 the critical performance indicator.

| Metric | standard_rf | sequential_rf | sequential_rf_all |
|---|---:|---:|---:|
| f1 | 0.3639 | 0.4019 | **0.4573** |
| recall | 0.3375 | 0.4225 | **0.5150** |
| precision | **0.3947** | 0.3832 | 0.4112 |
| oob_test_gap | **0.0034** | 0.0394 | 0.0230 |

**Training Times:**
- standard_rf (unweighted, avg_u): **2 seconds**
- sequential_rf (unweighted, avg_u): **5 minutes**
- sequential_rf_all (unweighted, max_samples=1.0): **30 minutes 42 seconds**

##### Meta-Labeling Strategy Analysis:

**sequential_rf_all is unequivocally the optimal choice** for this MA crossover meta-labeling strategy, despite the 6x longer training time. Here's the strategic justification:

1. **Transformative F1 Performance**: The F1 improvement is not incremental but **game-changing**:
   - +25.6% over standard_rf (0.3639 → 0.4573)
   - +13.8% over sequential_rf (0.4019 → 0.4573)
   - In meta-labeling, this level of improvement can dramatically boost strategy Sharpe ratio and reduce false entries

2. **Massive Recall Advantage**: The recall improvement is even more compelling:
   - +52.6% over standard_rf
   - +21.9% over sequential_rf
   - For meta-labeling, high recall means capturing more profitable secondary signals from your primary strategy

3. **Training Time Tradeoff is Justified**: While sequential_rf_all takes 6x longer (5 min vs 31 min), this is **absolutely acceptable** because:
   - Meta-labeling models are typically retrained infrequently (weekly/monthly)
   - The performance gains directly impact trading profitability
   - 31 minutes is reasonable for a production model that will be deployed for extended periods

4. **Overfitting Analysis Reconsidered**: 
   - sequential_rf_all actually shows **better generalization** than sequential_rf (OOB gap: 0.0230 vs 0.0394)
   - The "all" variant provides inherent regularization in this case
   - The moderate OOB gap is an acceptable tradeoff for the performance gains

##### Strategic Recommendation:

**Deploy sequential_rf_all** and structure your workflow accordingly:

- **Research Phase**: Use sequential_rf (5 min) for rapid prototyping and feature selection
- **Production Deployment**: Use sequential_rf_all (31 min) for final models
- **Retraining Schedule**: Batch retrain weekly/monthly to amortize the computational cost

The **performance differential is too substantial to ignore** for a meta-labeling application. The 26-minute additional training time is a trivial cost compared to the potential improvement in trading strategy performance.

**Bottom Line**: In meta-labeling, where F1 and recall directly determine your edge in filtering primary strategy signals, the 13.8% F1 improvement from sequential_rf_all is well worth the 6x training time increase. This is not a marginal gain but a **strategic advantage**.